## Fit IGM nuisance parameters to eBOSS P1D, one z at a time

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import time, os, sys
import matplotlib.pyplot as plt

# our own modules
from lace.emulator.emulator_manager import set_emulator
from cup1d.p1ds import (
    data_Chabanier2019,
    data_Karacayli2022
)
from cup1d.likelihood import lya_theory, likelihood, emcee_sampler
from cup1d.likelihood.sampler_pipeline import set_archive, set_P1D, set_P1D_hires, set_fid_cosmo, set_like
from cup1d.likelihood.input_pipeline import Args

In [2]:
# args = Args(emulator_label="Pedersen21")
args = Args(emulator_label="Pedersen23_ext8", training_set="Cabayol23")
# args = Args(emulator_label="Cabayol23+", training_set="Cabayol23")
# args = Args(emulator_label="Nyx_alphap", training_set="Nyx23_Oct2023")

In [3]:
args.data_label = "Chabanier19"
args.data_label_hires = "Karacayli22"
args.add_hires = True
args.add_metals = True
# development
args.n_igm=1
args.n_steps=2000
args.n_burn_in=400
args.parallel=False
args.explore=False

In [4]:
z_min=2.2
z_max=4.4
zs=np.arange(z_min,z_max+0.01,0.2)
Nz=len(zs)
print(Nz, zs)
# how many zs to plot
plot_every_iz=2

12 [2.2 2.4 2.6 2.8 3.  3.2 3.4 3.6 3.8 4.  4.2 4.4]


### Set archive

In [5]:
archive = set_archive(args.training_set)

### Set emulator

In [6]:
emulator = set_emulator(
    emulator_label=args.emulator_label,
    archive=archive,
)

Use custom archive provided by the user
Select emulator in Pedersen23_ext8
Gaussian Process emulator predicting the P1D, fitting coefficients to a 5th degree polynomial. It goes to scales of 8 Mpc^{-1} and z<=4.5. The parameters passed to the emulator will be overwritten to match these ones


 /Users/afont/Codes/igmhub/LaCE/lace/emulator/gp_emulator.py:266: UserWarning:Enforce average=both in training of GP emulator
 /Users/afont/Codes/igmhub/LaCE/lace/emulator/gp_emulator.py:268: UserWarning:Enforce val_scalinge=1 in training of GP emulator


Training GP on 330 points
GPs optimised in 0.18 seconds


 /opt/miniconda3/envs/cup1d/lib/python3.11/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


### Set fiducial cosmology

In [7]:
cosmo_fid = set_fid_cosmo(cosmo_label=args.cosmo_label)

### Set likelihoods (one per redshift bin)

In [8]:
def get_onez_likelihood(z):
    data, true_sim_igm = set_P1D(
        archive,
        emulator,
        args.data_label,
        cosmo_fid,
        cov_label=args.cov_label,
        apply_smoothing=False,
        z_min=z-0.01,
        z_max=z+0.01,
    )

    extra_data = set_P1D_hires(
        archive,
        emulator,
        cosmo_fid,
        args.data_label_hires,
        args.cov_label_hires,
        apply_smoothing=False,
        z_min=z-0.01,
        z_max=z+0.01,
    )
    
    like = set_like(
        emulator,
        data,
        extra_data,
        true_sim_igm,
        args.igm_label,
        1,
        cosmo_fid,
        fix_cosmo=True,
        add_metals=args.add_metals,
    )
    
    return like

In [9]:
allz_runs=[]
for iz,z in enumerate(zs):
    like = get_onez_likelihood(z)
    onez_run={'z':z, 'like':like}
    allz_runs.append(onez_run)

Nz = 14 , Nk = 13
Using 1 parameters for IGM model
free parameters ['ln_tau_0', 'ln_sigT_kms_0', 'ln_gamma_0', 'ln_kF_0', 'ln_SiIII_0']
metal tag ln_SiIII
will not store truth, working with real data
will not store truth, working with real data
Nz = 14 , Nk = 13
Using 1 parameters for IGM model
free parameters ['ln_tau_0', 'ln_sigT_kms_0', 'ln_gamma_0', 'ln_kF_0', 'ln_SiIII_0']
metal tag ln_SiIII
will not store truth, working with real data
will not store truth, working with real data
Nz = 14 , Nk = 13
Using 1 parameters for IGM model
free parameters ['ln_tau_0', 'ln_sigT_kms_0', 'ln_gamma_0', 'ln_kF_0', 'ln_SiIII_0']
metal tag ln_SiIII
will not store truth, working with real data
will not store truth, working with real data
Nz = 14 , Nk = 13
Using 1 parameters for IGM model
free parameters ['ln_tau_0', 'ln_sigT_kms_0', 'ln_gamma_0', 'ln_kF_0', 'ln_SiIII_0']
metal tag ln_SiIII
will not store truth, working with real data
will not store truth, working with real data
Nz = 14 , Nk = 13
Us

In [10]:
if False:
#for iz in range(Nz):
    plt.figure()
    allz_runs[iz]['like'].plot_p1d(residuals=False)
    allz_runs[iz]['like'].extra_p1d_like.plot_p1d(residuals=False)
    plt.xscale('log')

In [11]:
def fit_onez_likelihood(like, initial_values=None, run_sampler=False):
    def log_prob(theta):
        return log_prob.sampler.like.log_prob_and_blobs(theta)

    def set_log_prob(sampler):
        log_prob.sampler = sampler
        return log_prob

    sampler = emcee_sampler.EmceeSampler(
        like=like,
        rootdir=".",
        save_chain=False,
        nburnin=args.n_burn_in,
        nsteps=args.n_steps,
        parallel=args.parallel,
        explore=args.explore,
        fix_cosmology=args.fix_cosmo,
    )
    _log_prob = set_log_prob(sampler)

    if run_sampler:
        _emcee_sam = sampler.run_sampler(log_func=_log_prob)
        ind = np.argmax(sampler.lnprob.reshape(-1))
        nparam = sampler.chain.shape[-1]
        initial_values = sampler.chain.reshape(-1, nparam)[ind, :]
        print('MLE from chain will be initial value in minimizer',initial_values)

    npar=len(like.free_params)
    if initial_values is None:
        initial_values=0.5*np.ones(npar)
    else:
        print('starting from point',initial_values)
    sampler.run_minimizer(log_func=_log_prob,p0=initial_values)
    
    return sampler

In [ ]:
# turn this on to use the best-fit from the previous z-bin as a starting point
recycle_best_fit=False
run_sampler=True
for iz in range(Nz):
    if recycle_best_fit and iz>0:
        initial_values=allz_runs[iz-1]['best_fit_values']
        print(initial_values)
    else:
        initial_values=None
    sampler = fit_onez_likelihood(allz_runs[iz]['like'],
                                  initial_values=initial_values,run_sampler=run_sampler)
    allz_runs[iz]['sampler']=sampler
    # store chi2 and Ndof
    allz_runs[iz]['best_fit_values']=sampler.mle_cube
    npar=len(allz_runs[iz]['best_fit_values'])
    # eBOSS chi2 / Nk
    allz_runs[iz]['eB_chi2']=sampler.like.get_chi2(values=allz_runs[iz]['best_fit_values'])
    allz_runs[iz]['eB_Nk']=len(sampler.like.data.k_kms[0])
    # eBOSS chi2 / Nk
    allz_runs[iz]['hr_chi2']=sampler.like.extra_p1d_like.get_chi2(values=allz_runs[iz]['best_fit_values'])
    allz_runs[iz]['hr_Nk']=len(sampler.like.extra_p1d_like.data.k_kms[0])

Created directory: ./chain_74
setup with  1  ranks,  201  walkers, and  2000  steps
combined steps  482400 (should be close to  480000 )
set 201 walkers with 5 dimensions


 /Users/afont/Codes/igmhub/LaCE/lace/emulator/gp_emulator.py:550: UserWarning:Some of the requested k's are lower than the minimum training value k=0.09308422677303091


Step 500 out of 2400 
Step 600 out of 2400 
Step 700 out of 2400 
Step 800 out of 2400 
Step 900 out of 2400 
Step 1000 out of 2400 
Step 1100 out of 2400 
Step 1200 out of 2400 


In [ ]:
if False:
#for iz in range(Nz):
    sampler=allz_runs[iz]['sampler']
    
    best_fit_values=sampler.mle_cube
    npar=len(best_fit_values)
    chi2=sampler.like.get_chi2(values=best_fit_values)
    dof=len(sampler.like.data.k_kms[0])-npar
    f, ax = plt.subplots()

    sampler.like.plot_p1d(residuals=False,values=best_fit_values)
#    allz_runs[iz]['like'].plot_p1d(residuals=False,values=0.5*np.ones_like(sampler.mle_cube))
    sampler.like.extra_p1d_like.plot_p1d(residuals=False,values=best_fit_values)
#    allz_runs[iz]['like'].extra_p1d_like.plot_p1d(residuals=False,values=0.5*np.ones_like(sampler.mle_cube))
    plt.xscale('log')
    plt.text(0.5,0.1,'chi2={:.2f}, dof={}'.format(chi2,dof),transform=ax.transAxes)

    plt.savefig('p1d_eboss_highres_z{}.png'.format(zs[iz]))

In [ ]:
if False:
#for iz in range(Nz):
    sampler=allz_runs[iz]['sampler']
    plt.figure()
    allz_runs[iz]['like'].plot_p1d(residuals=True,values=sampler.mle_cube)
#    allz_runs[iz]['like'].plot_p1d(residuals=False,values=0.5*np.ones_like(sampler.mle_cube))
    allz_runs[iz]['like'].extra_p1d_like.plot_p1d(residuals=True,values=sampler.mle_cube)
#    allz_runs[iz]['like'].extra_p1d_like.plot_p1d(residuals=False,values=0.5*np.ones_like(sampler.mle_cube))
    plt.xscale('log')
    plt.savefig('res_p1d_eboss_highres_z{}.png'.format(zs[iz]))

### Look at the chi2 in each redshift

In [ ]:
for iz in range(Nz):
    run=allz_runs[iz]
    npar=len(run['best_fit_values'])
    print('z = {:.1f}, chi2 (eBOSS = {:.2f}, highres = {:.2f}, total = {:.2f}) , Ndof = {}'.format(
            zs[iz], run['eB_chi2'], run['hr_chi2'], run['eB_chi2']+run['hr_chi2'], 
            run['eB_Nk']+run['hr_Nk']-npar))

In [ ]:
eB_chi2=np.array([allz_runs[iz]['eB_chi2'] for iz in range(Nz)])
hr_chi2=np.array([allz_runs[iz]['hr_chi2'] for iz in range(Nz)])
eB_Nk=np.array([allz_runs[iz]['eB_Nk'] for iz in range(Nz)])
hr_Nk=np.array([allz_runs[iz]['hr_Nk'] for iz in range(Nz)])
npar=[len(allz_runs[iz]['best_fit_values']) for iz in range(Nz)]
plt.plot(zs,eB_chi2,label=r'eBOSS $\chi^2$',color='gray',ls=':')
plt.plot(zs,hr_chi2,label=r'highres $\chi^2$',color='gray',ls='-.')
plt.plot(zs,eB_chi2+hr_chi2,label=r'total $\chi^2$',color='red',ls='-')
plt.plot(zs,eB_Nk+hr_Nk-npar,label=r'total d.o.f.',color='red',ls='--')
plt.legend()
plt.xlabel('z')
plt.savefig('chi2_eboss_highres.png')

### Plot P1D measurements and fiducial theory models (eBOSS only)

In [ ]:
residuals=False
sampling_p1d=100
Nz=len(allz_runs)
# figure out y range for plot
ymin = 1e10
ymax = -1e10
for iz in range(0,Nz,plot_every_iz):
    col = plt.cm.jet(iz / (Nz - 1))
    yshift = iz / (Nz - 1)
    z=zs[iz]
    sampler=allz_runs[iz]['sampler']
    values=sampler.mle_cube
    values=0.5*np.ones_like(values)
    like=sampler.like
    if True:
        # get measured bins from data
        k_kms = like.data.k_kms[0]
        k_emu_kms = np.logspace(np.log10(min(k_kms)), np.log10(max(k_kms)), sampling_p1d)

        # ask emulator prediction for P1D in each bin
        emu_p1d = like.get_p1d_kms([k_emu_kms], values=values, return_covar=False)

        p1d_data = like.data.get_Pk_iz(0)
        p1d_cov = like.data.get_cov_iz(0)
        p1d_err = np.sqrt(np.diag(p1d_cov))
        p1d_theory = emu_p1d[0]

        if residuals:
            # interpolate theory to data kp values
            model = np.interp(k_kms, k_emu_kms, p1d_theory)
            plt.errorbar(
                k_kms,
                p1d_data / model + yshift,
                color=col,
                yerr=p1d_err / model,
                fmt="o",
                ms="4",
                label="z=" + str(np.round(z, 2)),
            )
            ymin = min(ymin, min(p1d_data / model + yshift))
            ymax = max(ymax, max(p1d_data / model + yshift))
            plt.plot(
                k_emu_kms,
                p1d_theory / p1d_theory + yshift,
                color=col,
                linestyle="dashed",
            )
        else:
            plt.errorbar(
                k_kms,
                p1d_data * k_kms / np.pi,
                color=col,
                yerr=p1d_err * k_kms / np.pi,
                fmt="o",
                ms="4",
                label="z=" + str(np.round(z, 2)),
            )
            plt.plot(
                k_emu_kms,
                (p1d_theory * k_emu_kms) / np.pi,
                color=col,
                linestyle="dashed",
            )
            ymin = min(ymin, min(p1d_data * k_kms / np.pi))
            ymax = max(ymax, max(p1d_data * k_kms / np.pi))

if residuals:
    plt.ylabel(r"$P_{\rm 1D}(z,k_\parallel)$ residuals")
    plt.ylim(ymin - 0.1, ymax + 0.1)
else:
    plt.yscale("log")
    plt.ylabel(r"$k_\parallel \, P_{\rm 1D}(z,k_\parallel) / \pi$")
    plt.ylim(0.8 * ymin, 1.2 * ymax)

plt.plot(-10, -10, linestyle="-", label="Data", color="k")
plt.plot(-10, -10, linestyle=":", label="Fiducial model", color="k")
plt.legend()
plt.xlabel(r"$k_\parallel$ [s/km]")
plt.xlim(min(k_kms) - 0.001, max(k_kms) + 0.001)
plt.tight_layout()
plt.savefig('p1d_eboss_fiducial.png')

### Plot P1D measurements and fiducial theory models (include high-res)

In [ ]:
residuals=False
sampling_p1d=100
Nz=len(allz_runs)
# figure out y range for plot
ymin = 1e10
ymax = -1e10
xmin = ymin
xmax = ymax
for iz in range(0,Nz,plot_every_iz):
    col = plt.cm.jet(iz / (Nz - 1))
    yshift = iz / (Nz - 1)
    z=zs[iz]
    sampler=allz_runs[iz]['sampler']
    values=sampler.mle_cube
    values=0.5*np.ones_like(values)
    like=sampler.like
    for highres in [False, True]:
        if highres:
            data = like.extra_p1d_like.data
            label = None
        else:
            data = like.data
            label="z=" + str(np.round(z, 2))
        # get measured bins from data
        k_kms = data.k_kms[0]
        k_emu_kms = np.logspace(np.log10(min(k_kms)), np.log10(max(k_kms)), sampling_p1d)

        # ask emulator prediction for P1D in each bin
        emu_p1d = like.get_p1d_kms([k_emu_kms], values=values, return_covar=False)

        p1d_data = data.get_Pk_iz(0)
        p1d_cov = data.get_cov_iz(0)
        p1d_err = np.sqrt(np.diag(p1d_cov))
        p1d_theory = emu_p1d[0]

        if residuals:
            # interpolate theory to data kp values
            model = np.interp(k_kms, k_emu_kms, p1d_theory)
            plt.errorbar(
                k_kms,
                p1d_data / model + yshift,
                color=col,
                yerr=p1d_err / model,
                fmt="o",
                ms="4",
                label=label,
            )
            ymin = min(ymin, min(p1d_data / model + yshift))
            ymax = max(ymax, max(p1d_data / model + yshift))
            plt.plot(
                k_emu_kms,
                p1d_theory / p1d_theory + yshift,
                color=col,
                linestyle="dashed",
            )
        else:
            plt.errorbar(
                k_kms,
                p1d_data * k_kms / np.pi,
                color=col,
                yerr=p1d_err * k_kms / np.pi,
                fmt="o",
                ms="4",
                label=label,
            )
            plt.plot(
                k_emu_kms,
                (p1d_theory * k_emu_kms) / np.pi,
                color=col,
                linestyle="dashed",
            )
            ymin = min(ymin, min(p1d_data * k_kms / np.pi))
            ymax = max(ymax, max(p1d_data * k_kms / np.pi))
            xmin = min(xmin, min(k_kms))
            xmax = max(xmax, max(k_kms))

if residuals:
    plt.ylabel(r"$P_{\rm 1D}(z,k_\parallel)$ residuals")
    plt.ylim(ymin - 0.1, ymax + 0.1)
else:
    plt.yscale("log")
    plt.ylabel(r"$k_\parallel \, P_{\rm 1D}(z,k_\parallel) / \pi$")
    plt.ylim(0.8 * ymin, 1.2 * ymax)

#plt.plot(-10, -10, linestyle="-", label="Data", color="k")
plt.plot(-10, -10, linestyle=":", label="Fiducial model", color="k")
plt.legend()
plt.xlabel(r"$k_\parallel$ [s/km]")
plt.xlim(0.9*xmin, 1.1*xmax)
plt.xscale('log')
plt.tight_layout()
plt.savefig('p1d_eboss_highres_fiducial.png')

### Plot P1D measurements and best-fit theory models (only eBOSS)

In [ ]:
residuals=False
sampling_p1d=100
Nz=len(allz_runs)
# figure out y range for plot
ymin = 1e10
ymax = -1e10
for iz in range(0,Nz,plot_every_iz):
    col = plt.cm.jet(iz / (Nz - 1))
    yshift = iz / (Nz - 1)
    z=zs[iz]
    sampler=allz_runs[iz]['sampler']
    values=sampler.mle_cube
    like=sampler.like
    if True:
        # get measured bins from data
        k_kms = like.data.k_kms[0]
        k_emu_kms = np.logspace(np.log10(min(k_kms)), np.log10(max(k_kms)), sampling_p1d)



        # ask emulator prediction for P1D in each bin
        emu_p1d = like.get_p1d_kms([k_emu_kms], values=values, return_covar=False)

        p1d_data = like.data.get_Pk_iz(0)
        p1d_cov = like.data.get_cov_iz(0)
        p1d_err = np.sqrt(np.diag(p1d_cov))
        p1d_theory = emu_p1d[0]

        if residuals:
            # interpolate theory to data kp values
            model = np.interp(k_kms, k_emu_kms, p1d_theory)
            plt.errorbar(
                k_kms,
                p1d_data / model + yshift,
                color=col,
                yerr=p1d_err / model,
                fmt="o",
                ms="4",
                label="z=" + str(np.round(z, 2)),
            )
            ymin = min(ymin, min(p1d_data / model + yshift))
            ymax = max(ymax, max(p1d_data / model + yshift))
            plt.plot(
                k_emu_kms,
                p1d_theory / p1d_theory + yshift,
                color=col,
                linestyle="dashed",
            )
        else:
            plt.errorbar(
                k_kms,
                p1d_data * k_kms / np.pi,
                color=col,
                yerr=p1d_err * k_kms / np.pi,
                fmt="o",
                ms="4",
                label="z=" + str(np.round(z, 2)),
            )
            plt.plot(
                k_emu_kms,
                (p1d_theory * k_emu_kms) / np.pi,
                color=col,
                linestyle="dashed",
            )
            ymin = min(ymin, min(p1d_data * k_kms / np.pi))
            ymax = max(ymax, max(p1d_data * k_kms / np.pi))

if residuals:
    plt.ylabel(r"$P_{\rm 1D}(z,k_\parallel)$ residuals")
    plt.ylim(ymin - 0.1, ymax + 0.1)
else:
    plt.yscale("log")
    plt.ylabel(r"$k_\parallel \, P_{\rm 1D}(z,k_\parallel) / \pi$")
    plt.ylim(0.8 * ymin, 1.2 * ymax)

plt.plot(-10, -10, linestyle="-", label="Data", color="k")
plt.plot(-10, -10, linestyle=":", label="Best-Fit model", color="k")
plt.legend()
plt.xlabel(r"$k_\parallel$ [s/km]")
plt.xlim(min(k_kms) - 0.001, max(k_kms) + 0.001)
plt.tight_layout()
plt.savefig('p1d_eboss_bestfit.png')

### Plot P1D measurements and best-fit theory models (include high-res)

In [ ]:
residuals=False
sampling_p1d=100
Nz=len(allz_runs)
# figure out y range for plot
ymin = 1e10
ymax = -1e10
xmin = ymin
xmax = ymax
for iz in range(0,Nz,plot_every_iz):
    col = plt.cm.jet(iz / (Nz - 1))
    yshift = iz / (Nz - 1)
    z=zs[iz]
    sampler=allz_runs[iz]['sampler']
    values=sampler.mle_cube
    #values=0.5*np.ones_like(values)
    like=sampler.like
    for highres in [False, True]:
        if highres:
            data = like.extra_p1d_like.data
            label = None
        else:
            data = like.data
            label="z=" + str(np.round(z, 2))
        # get measured bins from data
        k_kms = data.k_kms[0]
        k_emu_kms = np.logspace(np.log10(min(k_kms)), np.log10(max(k_kms)), sampling_p1d)

        # ask emulator prediction for P1D in each bin
        emu_p1d = like.get_p1d_kms([k_emu_kms], values=values, return_covar=False)

        p1d_data = data.get_Pk_iz(0)
        p1d_cov = data.get_cov_iz(0)
        p1d_err = np.sqrt(np.diag(p1d_cov))
        p1d_theory = emu_p1d[0]

        if residuals:
            # interpolate theory to data kp values
            model = np.interp(k_kms, k_emu_kms, p1d_theory)
            plt.errorbar(
                k_kms,
                p1d_data / model + yshift,
                color=col,
                yerr=p1d_err / model,
                fmt="o",
                ms="4",
                label=label,
            )
            ymin = min(ymin, min(p1d_data / model + yshift))
            ymax = max(ymax, max(p1d_data / model + yshift))
            plt.plot(
                k_emu_kms,
                p1d_theory / p1d_theory + yshift,
                color=col,
                linestyle="dashed",
            )
        else:
            plt.errorbar(
                k_kms,
                p1d_data * k_kms / np.pi,
                color=col,
                yerr=p1d_err * k_kms / np.pi,
                fmt="o",
                ms="4",
                label=label,
            )
            plt.plot(
                k_emu_kms,
                (p1d_theory * k_emu_kms) / np.pi,
                color=col,
                linestyle="dashed",
            )
            ymin = min(ymin, min(p1d_data * k_kms / np.pi))
            ymax = max(ymax, max(p1d_data * k_kms / np.pi))
            xmin = min(xmin, min(k_kms))
            xmax = max(xmax, max(k_kms))

if residuals:
    plt.ylabel(r"$P_{\rm 1D}(z,k_\parallel)$ residuals")
    plt.ylim(ymin - 0.1, ymax + 0.1)
else:
    plt.yscale("log")
    plt.ylabel(r"$k_\parallel \, P_{\rm 1D}(z,k_\parallel) / \pi$")
    plt.ylim(0.8 * ymin, 1.2 * ymax)

#plt.plot(-10, -10, linestyle="-", label="Data", color="k")
plt.plot(-10, -10, linestyle=":", label="Best-fit model", color="k")
plt.legend()
plt.xlabel(r"$k_\parallel$ [s/km]")
plt.xlim(0.9*xmin, 1.1*xmax)
plt.xscale('log')
plt.tight_layout()
plt.savefig('p1d_eboss_highres_bestfit.png')

### Let's take a look at the redshift evolution of best-fit parameters

In [ ]:
mF=[]
T0=[]
gamma=[]
sigT_kms=[]
kF_kms=[]
f_SiIII=[]
for iz,z in enumerate(zs):
    run=allz_runs[iz]
    like=run['like']
    best_fit_values=run['best_fit_values']
    like_params=like.parameters_from_sampling_point(best_fit_values)
    F_model = like.theory.F_model_fid.get_new_model(like_params)
    T_model = like.theory.T_model_fid.get_new_model(like_params)
    P_model = like.theory.P_model_fid.get_new_model(like_params)
    X_model = like.theory.metal_models[0].get_new_model(like_params)
    mF.append(F_model.get_mean_flux(z=z))
    T0.append(T_model.get_T0(z=z))
    sigT_kms.append(T_model.get_sigT_kms(z=z))
    gamma.append(T_model.get_gamma(z=z))
    kF_kms.append(P_model.get_kF_kms(z=z))
    f_SiIII.append(X_model.get_amplitude(z=z))

In [ ]:
plt.plot(zs,mF)
plt.xlabel('z')
plt.ylabel('<F>')
plt.savefig('mF_eboss_highres.png')

In [ ]:
plt.plot(zs,T0)
plt.xlabel('z')
plt.ylabel(r'$T_0 [K]$')
plt.savefig('T0_eboss_highres.png')

In [ ]:
plt.plot(zs,gamma)
plt.xlabel('z')
plt.ylabel(r'$\gamma$')
plt.savefig('gamma_eboss_highres.png')

In [ ]:
plt.plot(zs,kF_kms)
plt.xlabel('z')
plt.ylabel(r'$k_F$ [s/km]')
plt.savefig('kF_eboss_highres.png')

In [ ]:
plt.plot(zs,f_SiIII)
plt.xlabel('z')
plt.ylabel(r'$f_{\rm SiIII}$')
plt.savefig('fSiIII_eboss_highres.png')

### Now let's use MCMC mean and rms

In [ ]:
mF=np.zeros([Nz,6])
T0=np.zeros([Nz,6])
gamma=np.zeros([Nz,6])
kF_kms=np.zeros([Nz,6])
f_SiIII=np.zeros([Nz,6])
for iz,z in enumerate(zs):
    run=allz_runs[iz]
    like=run['like']
    chain=run['sampler'].chain
    mean=chain.mean(axis=(0,1))
    rms=np.sqrt(chain.var(axis=(0,1)))
    for i in range(3):
        like_params=like.parameters_from_sampling_point(mean+(i-1)*rms)
        F_model = like.theory.F_model_fid.get_new_model(like_params)
        T_model = like.theory.T_model_fid.get_new_model(like_params)
        P_model = like.theory.P_model_fid.get_new_model(like_params)
        X_model = like.theory.metal_models[0].get_new_model(like_params)
        mF[iz,i]=F_model.get_mean_flux(z=z)
        T0[iz,i]=T_model.get_T0(z=z)
        gamma[iz,i]=T_model.get_gamma(z=z)
        kF_kms[iz,i]=P_model.get_kF_kms(z=z)
        f_SiIII[iz,i]=X_model.get_amplitude(z=z)
    for i in range(3):
        p=10+i*40
        pp=np.percentile(chain,p,axis=(0,1))
        like_params=like.parameters_from_sampling_point(pp)
        F_model = like.theory.F_model_fid.get_new_model(like_params)
        T_model = like.theory.T_model_fid.get_new_model(like_params)
        P_model = like.theory.P_model_fid.get_new_model(like_params)
        X_model = like.theory.metal_models[0].get_new_model(like_params)
        mF[iz,i+3]=F_model.get_mean_flux(z=z)
        T0[iz,i+3]=T_model.get_T0(z=z)
        gamma[iz,i+3]=T_model.get_gamma(z=z)
        kF_kms[iz,i+3]=P_model.get_kF_kms(z=z)
        f_SiIII[iz,i+3]=X_model.get_amplitude(z=z)

In [ ]:
plt.fill_between(zs,mF[:,3],mF[:,5],alpha=0.3)
plt.plot(zs,mF[:,4])
plt.xlabel('z')
plt.ylabel('<F>')
plt.savefig('mF_eboss_highres.png')

In [ ]:
plt.fill_between(zs,T0[:,3],T0[:,5],alpha=0.3)
plt.plot(zs,T0[:,4])
plt.xlabel('z')
plt.ylabel(r'$T_0 [K]$')
plt.savefig('T0_eboss_highres.png')

In [ ]:
plt.fill_between(zs,gamma[:,3],gamma[:,5],alpha=0.3)
plt.plot(zs,gamma[:,4])
plt.xlabel('z')
plt.ylabel(r'$\gamma$')
plt.savefig('gamma_eboss_highres.png')

In [ ]:
plt.fill_between(zs,kF_kms[:,3],kF_kms[:,5],alpha=0.3)
plt.plot(zs,kF_kms[:,4])
plt.xlabel('z')
plt.ylabel(r'$k_F$ [s/km]')
plt.savefig('kF_eboss_highres.png')

In [ ]:
plt.fill_between(zs,f_SiIII[:,3],f_SiIII[:,5],alpha=0.3)
plt.plot(zs,f_SiIII[:,4])
plt.xlabel('z')
plt.ylabel(r'$f_{\rm SiIII}$')
plt.savefig('fSiIII_eboss_highres.png')